In [1]:
import pandas as pd
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle

In [2]:
df = pd.read_hdf('predict.h5')

In [4]:
#Generate scaler
df_ori = df.copy(deep=True)
df_ori.drop('geohash6',axis=1,inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(df_ori)
pickle.dump(scaler, open("scaler.pkl", "wb"))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
def createScaledAndDf(start,end):
    dfx = df.copy(deep=True)
    dfx.drop('geohash6',axis=1,inplace=True)
    dfx = dfx.loc[(dfx['day']>=start) & (dfx['day']<=end)]
    dfx = dfx.astype(np.float32)
    dfx = dfx.fillna(0)
    scaledx = scaler.transform(dfx)
    print(scaledx.shape)
    return scaledx,dfx
def createTraining(idx,scaledx):
    print(scaledx.shape)
    trainx = scaledx[:idx, :]
    testx = scaledx[idx+1:, :]
    x_trainx, y_trainx = trainx[:, :-1], trainx[:, -1]
    x_testx, y_testx = testx[:, :-1], testx[:, -1]
    print(x_trainx.shape, y_trainx.shape, x_testx.shape, y_testx.shape)
    return (x_trainx, y_trainx, x_testx, y_testx)
def pred(x_trainx, y_trainx, x_testx, y_testx,random_s):
    model = XGBRegressor(objective='reg:squarederror',learning_rate=0.035,n_estimators=500,random_state=random_s,max_depth=9
                         ,min_child_weight=5,nthread=-1)
    eval_set = [(x_trainx, y_trainx), (x_testx, y_testx)]
    model.fit(x_trainx, y_trainx, eval_metric="rmse",early_stopping_rounds=20, eval_set=eval_set, verbose=True)
    return model

In [6]:

def pred_sil(x_trainx, y_trainx, x_testx, y_testx,random_s,n_est):
    model = XGBRegressor(objective='reg:squarederror',learning_rate=0.035,n_estimators=n_est,random_state=random_s,max_depth=9,min_child_weight=5,nthread=-1)
    model.fit(x_trainx, y_trainx, eval_metric="rmse", verbose=True)
    return model

In [7]:
import time
def timeStart():
    start = time.time()
    return start
def timeEnd():
    end = time.time()
    print("Total time taken: {} min".format((end - start)/60))

In [15]:
import os
if not os.path.exists('model'):
    os.makedirs('model')

In [11]:
start = timeStart()
import random
days = df['day'].unique()
i = 1
for count, element in enumerate(days, days[0]):
    if count == 11:
        print(df.loc[(df.day >= element) & (df.day <=element+3)].shape)
        print('training for {} to {} days..'.format(element,element+3))
        print('saving to file{}'.format(i))
        print('last file, breaking')
        scaled1,df1 = createScaledAndDf(element,element+4)
        df1.reset_index(0,drop=True,inplace=True)
        train_idx1 = df1.loc[df1['day']<element+4].index[-1]
        rands = random.randint(1,100000)        
        x_train1, y_train1 , x_test1, y_test1 = createTraining(train_idx1,scaled1)
        model1 = pred_sil(x_train1, y_train1, x_test1, y_test1,rands,250)
        pickle.dump(model1, open("model/final{}.model.dat".format(i), "wb"))
        break
    if (count % 2 != 0):
        print(df.loc[(df.day >= element) & (df.day <=element+4)].shape)
        print('training for {} to {} days..'.format(element,element+4))
        print('saving to file{}'.format(i))
        scaled1,df1 = createScaledAndDf(element,element+5)
        df1.reset_index(0,drop=True,inplace=True)
        train_idx1 = df1.loc[df1['day']<element+5].index[-1]
        rands = random.randint(1,100000)        
        x_train1, y_train1 , x_test1, y_test1 = createTraining(train_idx1,scaled1)
        model1 = pred(x_train1, y_train1, x_test1, y_test1,rands)
        y_pred1 = model1.predict(x_test1)
        RMSE1 = sqrt(mean_squared_error(y_test1, y_pred1))
        print("RMS : %.2f%%" % (RMSE1 * 100.0))
        pickle.dump(model1, open("model/final{}.model.dat".format(i), "wb"))
        i+=1
timeEnd()

(347933, 45)
training for 1 to 5 days..
saving to file1
(413429, 44)
(413429, 44)
(347932, 43) (347932,) (65496, 43) (65496,)
[0]	validation_0-rmse:0.410607	validation_1-rmse:0.419654
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.396326	validation_1-rmse:0.405056
[2]	validation_0-rmse:0.382547	validation_1-rmse:0.390982
[3]	validation_0-rmse:0.369253	validation_1-rmse:0.377401
[4]	validation_0-rmse:0.356428	validation_1-rmse:0.364304
[5]	validation_0-rmse:0.344054	validation_1-rmse:0.351664
[6]	validation_0-rmse:0.332116	validation_1-rmse:0.339455
[7]	validation_0-rmse:0.320599	validation_1-rmse:0.327693
[8]	validation_0-rmse:0.309488	validation_1-rmse:0.316347
[9]	validation_0-rmse:0.298769	validation_1-rmse:0.305403
[10]	validation_0-rmse:0.288428	validation_1-rmse:0.294846
[11]	validation_0-rmse:0.278453	validation_1-rmse:0.284673
[12]	validation_0

[135]	validation_0-rmse:0.027388	validation_1-rmse:0.028854
[136]	validation_0-rmse:0.027359	validation_1-rmse:0.028842
[137]	validation_0-rmse:0.027331	validation_1-rmse:0.028827
[138]	validation_0-rmse:0.027305	validation_1-rmse:0.028816
[139]	validation_0-rmse:0.02728	validation_1-rmse:0.028806
[140]	validation_0-rmse:0.027252	validation_1-rmse:0.028797
[141]	validation_0-rmse:0.027226	validation_1-rmse:0.028786
[142]	validation_0-rmse:0.027201	validation_1-rmse:0.028778
[143]	validation_0-rmse:0.027178	validation_1-rmse:0.028767
[144]	validation_0-rmse:0.027159	validation_1-rmse:0.028762
[145]	validation_0-rmse:0.027138	validation_1-rmse:0.028754
[146]	validation_0-rmse:0.027119	validation_1-rmse:0.028748
[147]	validation_0-rmse:0.0271	validation_1-rmse:0.028742
[148]	validation_0-rmse:0.027081	validation_1-rmse:0.028735
[149]	validation_0-rmse:0.027063	validation_1-rmse:0.028729
[150]	validation_0-rmse:0.027046	validation_1-rmse:0.028722
[151]	validation_0-rmse:0.027023	validation

[62]	validation_0-rmse:0.053293	validation_1-rmse:0.054106
[63]	validation_0-rmse:0.051948	validation_1-rmse:0.05282
[64]	validation_0-rmse:0.050662	validation_1-rmse:0.051593
[65]	validation_0-rmse:0.049431	validation_1-rmse:0.050424
[66]	validation_0-rmse:0.048256	validation_1-rmse:0.049315
[67]	validation_0-rmse:0.047135	validation_1-rmse:0.048256
[68]	validation_0-rmse:0.046066	validation_1-rmse:0.047247
[69]	validation_0-rmse:0.045047	validation_1-rmse:0.046291
[70]	validation_0-rmse:0.044076	validation_1-rmse:0.04538
[71]	validation_0-rmse:0.04315	validation_1-rmse:0.044514
[72]	validation_0-rmse:0.042269	validation_1-rmse:0.043697
[73]	validation_0-rmse:0.041431	validation_1-rmse:0.042914
[74]	validation_0-rmse:0.040631	validation_1-rmse:0.042178
[75]	validation_0-rmse:0.039873	validation_1-rmse:0.041479
[76]	validation_0-rmse:0.039154	validation_1-rmse:0.040818
[77]	validation_0-rmse:0.038468	validation_1-rmse:0.040193
[78]	validation_0-rmse:0.037815	validation_1-rmse:0.039595


[200]	validation_0-rmse:0.02579	validation_1-rmse:0.030212
[201]	validation_0-rmse:0.025771	validation_1-rmse:0.030208
[202]	validation_0-rmse:0.02576	validation_1-rmse:0.030207
[203]	validation_0-rmse:0.025754	validation_1-rmse:0.030205
[204]	validation_0-rmse:0.025738	validation_1-rmse:0.030204
[205]	validation_0-rmse:0.02572	validation_1-rmse:0.030202
[206]	validation_0-rmse:0.02571	validation_1-rmse:0.030199
[207]	validation_0-rmse:0.025694	validation_1-rmse:0.030199
[208]	validation_0-rmse:0.025676	validation_1-rmse:0.030196
[209]	validation_0-rmse:0.025669	validation_1-rmse:0.030195
[210]	validation_0-rmse:0.025649	validation_1-rmse:0.03019
[211]	validation_0-rmse:0.025637	validation_1-rmse:0.03019
[212]	validation_0-rmse:0.025626	validation_1-rmse:0.030188
[213]	validation_0-rmse:0.025609	validation_1-rmse:0.030183
[214]	validation_0-rmse:0.025598	validation_1-rmse:0.030182
[215]	validation_0-rmse:0.025589	validation_1-rmse:0.030181
[216]	validation_0-rmse:0.025577	validation_1-

[338]	validation_0-rmse:0.024559	validation_1-rmse:0.030092
[339]	validation_0-rmse:0.024554	validation_1-rmse:0.030091
[340]	validation_0-rmse:0.024547	validation_1-rmse:0.030091
[341]	validation_0-rmse:0.024542	validation_1-rmse:0.030091
[342]	validation_0-rmse:0.024539	validation_1-rmse:0.030091
[343]	validation_0-rmse:0.024534	validation_1-rmse:0.030091
[344]	validation_0-rmse:0.024529	validation_1-rmse:0.03009
[345]	validation_0-rmse:0.024524	validation_1-rmse:0.03009
[346]	validation_0-rmse:0.024516	validation_1-rmse:0.030089
[347]	validation_0-rmse:0.024509	validation_1-rmse:0.03009
[348]	validation_0-rmse:0.024504	validation_1-rmse:0.03009
[349]	validation_0-rmse:0.024494	validation_1-rmse:0.03009
[350]	validation_0-rmse:0.024492	validation_1-rmse:0.03009
[351]	validation_0-rmse:0.024486	validation_1-rmse:0.03009
[352]	validation_0-rmse:0.024479	validation_1-rmse:0.03009
[353]	validation_0-rmse:0.024476	validation_1-rmse:0.03009
[354]	validation_0-rmse:0.02447	validation_1-rmse

...

[128]	validation_0-rmse:0.02647	validation_1-rmse:0.029142
[129]	validation_0-rmse:0.026426	validation_1-rmse:0.029118
[130]	validation_0-rmse:0.026388	validation_1-rmse:0.029095
[131]	validation_0-rmse:0.02635	validation_1-rmse:0.029072
[132]	validation_0-rmse:0.026314	validation_1-rmse:0.029055
[133]	validation_0-rmse:0.026278	validation_1-rmse:0.029033
[134]	validation_0-rmse:0.026245	validation_1-rmse:0.029015
[135]	validation_0-rmse:0.02621	validation_1-rmse:0.028997
[136]	validation_0-rmse:0.026176	validation_1-rmse:0.028981
[137]	validation_0-rmse:0.026145	validation_1-rmse:0.028965
[138]	validation_0-rmse:0.026116	validation_1-rmse:0.028949
[139]	validation_0-rmse:0.026085	validation_1-rmse:0.028937
[140]	validation_0-rmse:0.026059	validation_1-rmse:0.028924
[141]	validation_0-rmse:0.026034	validation_1-rmse:0.028916
[142]	validation_0-rmse:0.026011	validation_1-rmse:0.028908
[143]	validation_0-rmse:0.025987	validation_1-rmse:0.028897
[144]	validation_0-rmse:0.025963	validation

[266]	validation_0-rmse:0.024573	validation_1-rmse:0.028659
[267]	validation_0-rmse:0.024562	validation_1-rmse:0.028657
[268]	validation_0-rmse:0.024555	validation_1-rmse:0.028656
[269]	validation_0-rmse:0.02455	validation_1-rmse:0.028656
[270]	validation_0-rmse:0.024548	validation_1-rmse:0.028656
[271]	validation_0-rmse:0.024539	validation_1-rmse:0.028657
[272]	validation_0-rmse:0.024535	validation_1-rmse:0.028658
[273]	validation_0-rmse:0.024523	validation_1-rmse:0.028657
[274]	validation_0-rmse:0.024519	validation_1-rmse:0.028656
[275]	validation_0-rmse:0.024502	validation_1-rmse:0.028654
[276]	validation_0-rmse:0.024495	validation_1-rmse:0.028653
[277]	validation_0-rmse:0.024491	validation_1-rmse:0.028652
[278]	validation_0-rmse:0.024483	validation_1-rmse:0.028651
[279]	validation_0-rmse:0.024468	validation_1-rmse:0.028651
[280]	validation_0-rmse:0.024462	validation_1-rmse:0.02865
[281]	validation_0-rmse:0.024454	validation_1-rmse:0.028649
[282]	validation_0-rmse:0.024448	validatio

[90]	validation_0-rmse:0.031615	validation_1-rmse:0.033486
[91]	validation_0-rmse:0.031286	validation_1-rmse:0.033186
[92]	validation_0-rmse:0.030978	validation_1-rmse:0.032911
[93]	validation_0-rmse:0.03069	validation_1-rmse:0.032647
[94]	validation_0-rmse:0.030416	validation_1-rmse:0.032401
[95]	validation_0-rmse:0.030159	validation_1-rmse:0.032166
[96]	validation_0-rmse:0.029913	validation_1-rmse:0.031948
[97]	validation_0-rmse:0.029684	validation_1-rmse:0.031747
[98]	validation_0-rmse:0.029465	validation_1-rmse:0.031552
[99]	validation_0-rmse:0.029264	validation_1-rmse:0.031374
[100]	validation_0-rmse:0.02907	validation_1-rmse:0.031206
[101]	validation_0-rmse:0.028887	validation_1-rmse:0.031055
[102]	validation_0-rmse:0.028715	validation_1-rmse:0.030906
[103]	validation_0-rmse:0.028552	validation_1-rmse:0.030768
[104]	validation_0-rmse:0.028397	validation_1-rmse:0.030637
[105]	validation_0-rmse:0.028254	validation_1-rmse:0.030517
[106]	validation_0-rmse:0.028117	validation_1-rmse:0

[228]	validation_0-rmse:0.024754	validation_1-rmse:0.028758
[229]	validation_0-rmse:0.024751	validation_1-rmse:0.028757
[230]	validation_0-rmse:0.024742	validation_1-rmse:0.028757
[231]	validation_0-rmse:0.024725	validation_1-rmse:0.028757
[232]	validation_0-rmse:0.024714	validation_1-rmse:0.028758
[233]	validation_0-rmse:0.024706	validation_1-rmse:0.028758
[234]	validation_0-rmse:0.02469	validation_1-rmse:0.028757
[235]	validation_0-rmse:0.024683	validation_1-rmse:0.028757
[236]	validation_0-rmse:0.024667	validation_1-rmse:0.028757
[237]	validation_0-rmse:0.024657	validation_1-rmse:0.028757
[238]	validation_0-rmse:0.024648	validation_1-rmse:0.028758
[239]	validation_0-rmse:0.024636	validation_1-rmse:0.028758
[240]	validation_0-rmse:0.024626	validation_1-rmse:0.028758
[241]	validation_0-rmse:0.024619	validation_1-rmse:0.028758
[242]	validation_0-rmse:0.024601	validation_1-rmse:0.028758
[243]	validation_0-rmse:0.024592	validation_1-rmse:0.028758
[244]	validation_0-rmse:0.024579	validati

[91]	validation_0-rmse:0.030289	validation_1-rmse:0.033329
[92]	validation_0-rmse:0.02997	validation_1-rmse:0.033077
[93]	validation_0-rmse:0.029668	validation_1-rmse:0.032841
[94]	validation_0-rmse:0.029384	validation_1-rmse:0.032619
[95]	validation_0-rmse:0.029113	validation_1-rmse:0.032413
[96]	validation_0-rmse:0.028857	validation_1-rmse:0.032219
[97]	validation_0-rmse:0.028615	validation_1-rmse:0.032036
[98]	validation_0-rmse:0.028388	validation_1-rmse:0.031862
[99]	validation_0-rmse:0.028173	validation_1-rmse:0.031705
[100]	validation_0-rmse:0.027972	validation_1-rmse:0.031557
[101]	validation_0-rmse:0.027784	validation_1-rmse:0.031418
[102]	validation_0-rmse:0.027604	validation_1-rmse:0.031288
[103]	validation_0-rmse:0.027433	validation_1-rmse:0.031165
[104]	validation_0-rmse:0.027272	validation_1-rmse:0.031054
[105]	validation_0-rmse:0.02712	validation_1-rmse:0.030948
[106]	validation_0-rmse:0.026977	validation_1-rmse:0.03085
[107]	validation_0-rmse:0.026841	validation_1-rmse:0